#Doc2Vec: 문서 임베딩 방법

전자공시시스템(Dart)의 회사들의 사업보고서를 학습시키고 사업 보고서가 유사한 회사들을 찾는 것을 만들자

In [1]:
!pip install --upgrade --no-cache-dir gdown
# dart.csv 파일 다운로드
!gdown https://drive.google.com/uc?id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL


  Attempting uninstall: gdown
    Found existing installation: gdown 4.7.3
    Uninstalling gdown-4.7.3:
      Successfully uninstalled gdown-4.7.3
Downloading...
From (original): https://drive.google.com/uc?id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL
From (redirected): https://drive.usercontent.google.com/download?id=1XS0UlE8gNNTRjnL6e64sMacOhtVERIqL&confirm=t&uuid=27b59e16-b6e3-4428-b33b-ead28f0f47a6
To: /content/dart.csv
100% 125M/125M [00:02<00:00, 61.0MB/s]


In [2]:
# 형태소 분석기 Mecab 설치
!pip install konlpy
!pip install mecab-python
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 41.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 30.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.7/581.7 kB 7.6 MB/s eta 0:00:00
  Created wheel for mecab-python: filename=mecab_python-1.0.0-py3-none-any.whl size=1232 sha256=a7df5fc5ba7e63bbe1a1829cadbbb404264630f19c9baaf17397f82e5b1cdc29
  Stored in directory: /root/.cache/pip/wheels/ec/41/c8/ce8fb469be2547ce596c4000613806505dc54c296aae45da71
Successfully built mecab-python
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0   835k      0  0:00:01  0:00:01 --:--:--  240M
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example

In [3]:
import pandas as pd
from konlpy.tag import Mecab
from gensim.models.doc2vec import TaggedDocument
from tqdm import tqdm


In [4]:
df = pd.read_csv('/content/dart.csv',  sep=',')
df = df.dropna()
df

,code,market,name,business
0,000020,KOSPI,동화약품,II. 사업의 내용\n1. 사업의 개요\n가. 일반적인 사항\n기업회계기준서 제11...
1,000040,KOSPI,KR모터스,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n수출주력시장인 유럽 불...
2,000050,KOSPI,경방,II. 사업의 내용\n1. 사업의 개요\n(1) 산업의 특성\n[섬유사업부문]\n면...
3,000060,KOSPI,메리츠화재,"II. 사업의 내용\n1. 사업의 개요\n가. 산업의 특성, 성장성, 경기변동의 특..."
4,000070,KOSPI,삼양홀딩스,"II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n지주회사(持株會社, H..."
...,...,...,...,...
2582,950130,KOSDAQ,엑세스바이오,II. 사업의 내용\n1. 사업의 개\n요\n당사는 체외진단 기술을 토대로 말라리아...
2583,950140,KOSDAQ,잉글우드랩,II. 사업의 내용\n1. 사업의 개요\n가. 화장품 ODM산업 개요\nODM은 O...
2584,950160,KOSDAQ,코오롱티슈진,II. 사업의 내용\n당사는 1999년 6월 9일 세포 유전자 기술 Platform...
2585,950170,KOSDAQ,JTC,II. 사업의 내용\n1. 사업의 개요\n가. 업계의 현황\n(1) 산업의 특성\n...


business 열에 사업보고서 내용이 들어있다. 이를 형태소 분석하고 Doc2Vec 학습을 시키기 위해 데이터의 형식을 변환하자. 문서의 제목(회사이름)과 본문을 뽑아내야 한다.

corpus의 각 element의 태그로는 회사 이름, 내용은 본문 내용의 단어 토큰화 결과를 넣자.

In [6]:
mecab = Mecab()

tagged_corpus_list = []

for index, row in tqdm(df.iterrows(), total=len(df)):
  text = row['business']
  tag = row['name']
  tagged_corpus_list.append(TaggedDocument(tags = [tag], words=mecab.morphs(text)))

print('문서의 수: ', len(tagged_corpus_list))


100%|██████████| 2295/2295 [02:05<00:00, 18.30it/s]

문서의 수:  2295


In [7]:
tagged_corpus_list[0]

TaggedDocument(words=['II', '.', '사업', '의', '내용', '1', '.', '사업', '의', '개요', '가', '.', '일반', '적', '인', '사항', '기업', '회계', '기준', '서', '제', '1110', '호', '"', '연결', '재무제표', '"', '의', '의하', '여', '2018', '년', '12', '월', '17', '일', '에', '설립', '한', '동화', '크립톤', '기업가', '정신', '제일', '호', '창업', '벤처', '전문', '사모', '투자', '합자회사', '를', '종속', '회사', '에', '편입', '하', '였', '습니다', '.', '나', '.', '지배', '기업', '의', '현황', '1', ')', '산업', '의', '특성', '제약', '산업', '은', '약학', ',', '화학', ',', '생물학', '등', '여러', '분야', '의', '지식', '과', '기술', '이', '복합', '적', '으로', '필요', '한', '기술', '·', '지식', '집약', '형', '산업', '으로', '인간', '의', '생명', '과', '보건', '에', '직접', '적', '으로', '관련', '된', '제품', '을', '생산', '하', '고', ',', '개발', '·', '제조', '·', '유통', '·', '가격', '등', '을', '정부', '에서', '엄격', '하', '게', '규제', '하', '고', '관리', '하', '는', '산업', '적', '특성', '을', '가지', '고', '있', '습니다', '.', '또한', '신약', '개발', '에', '는', '막대', '한', '시간', '과', '비용', '이', '소요', '되', '며', ',', '성공', '확률', '또한', '높', '지', '않', '으나', '신약', '개발', '에', '성공', '할', '경우', '엔', '엄청난

#모델 학습 / 테스트

만든 데이터로 문서를 임베딩 벡터로 잘 만드는 모델로 학습시키자

In [8]:
from gensim.models import doc2vec


In [9]:
model = doc2vec.Doc2Vec(vector_size=300, alpha=0.025, min_alpha=0.025, workers=8, window=8)

model.build_vocab(tagged_corpus_list)

model.train(tagged_corpus_list, total_examples = model.corpus_count, epochs=20)

model.save('dart.doc2vec')

In [10]:
similar_doc = model.dv.most_similar('카카오')
print(similar_doc)

[('네오위즈홀딩스', 0.5130359530448914), ('네오위즈', 0.5121862888336182), ('카카오게임즈', 0.49201005697250366), ('컴투스', 0.4852648079395294), ('NAVER', 0.4708130657672882), ('넥슨지티', 0.46961313486099243), ('엠게임', 0.46802258491516113), ('키이스트', 0.46755701303482056), ('케어랩스', 0.4674588143825531), ('액토즈소프트', 0.46448376774787903)]
